# MICrONS DataClean package: basic overview 

## Setup and data download

The package allows to download and easily process data from the MICrONS mm3 dataset. This Notebook shows the typical workflow to obtain the data for the first time, and how to work with it.

The first step is to include all the relevant libraries to work.

In [1]:
#Cell magic for autoreloading files
%reload_ext autoreload
%autoreload 2

#Import general packages
import pandas as pd

#Import our package
import microns_datacleaner as mic

The `MicronsDataCleaner` class is the main container of the library. It allows you to download the nucleus tables, merge them, and also to query the connections between neurons. To use it, we have to initialize the class.
The data will be downloaded in the `datadir` argument of the class constructor. Please note that the path is relative to the place where the code is executed (i.e. where this notebook is located, or the folder where `python` is invoked). 

One can indicate the version of the tables to download. Note that a subfolder is created to allocate all the tables for the specified version.

Finally, the `download_policy` argument states how many tables will be downloaded. The `minimum` one is the basic setup, which downloads only the necessary tables for the package to build the unit table and generate layer information. `all` downloads all the tables. 

In [31]:
cleaner = mic.MicronsDataCleaner(datadir = "../data", version=1300, download_policy='minimum') 

It is possible to set the mode to `extra` to indicate if you want not only the minimum, but also more tables. The name of the tables you want has to be included in the `extra_tables = [...]` argument. The argument only has effect if `download_policy=extra`. 


What tables are available for the version we are working with? Let's find out:

In [3]:
cleaner.get_table_list()

['baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'allen_column_mtypes_v2',
 'bodor_pt_cells',
 'aibs_metamodel_mtypes_v661_v2',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'apl_functional_coreg_vess_fwd',
 'vortex_compartment_targets',
 'baylor_log_reg_cell_type_coarse_v1',
 'functional_properties_v3_bcm',
 'gamlin_2023_mcs',
 'l5et_column',
 'pt_synapse_targets',
 'coregistration_manual_v4',
 'cg_cell_type_calls',
 'synapses_pni_2',
 'nucleus_detection_v0',
 'vortex_manual_nodes_of_ranvier',
 'vortex_astrocyte_proofreading_status',
 'bodor_pt_target_proofread',
 'nucleus_functional_area_assignment',
 'coregistration_auto_phase3_fwd_apl_vess_combined_v2',
 'synapse_target_structure',
 'coregistration_auto_phase3_fwd_v2',
 'gamlin_2023_mcs_met_types',
 'vortex_manual_myelination_v0',
 'proofreading_status_and_strategy',
 'synapse_target_predictions_ssa',
 'aibs_metamodel_celltypes_v661']

Let's assume that we have downloaded already the tables and we realise that we would like to download something else afterwards. Instead of redownloading everything using the `extra` mode, it is possible to ask the client to get the tables for you. Let's download the automatic functional coregistration table that we will use later in the tutorial:

In [4]:
cleaner.download_tables(['coregistration_auto_phase3_fwd_apl_vess_combined_v2'])

For many applications, the tables included in the `minimum` mode should be enough. The function `process_nucleus_data` will connects the downloaded `minimum` tables to the nucleus reference, generating a unified unit table with all the useful information for modelling. The resulting table includes

1. The `nucleus_id`, the global id reference that tables target to.
2. The `pt_root_id`, essential to work with synapses.
3. The `pt_position` (in 3D, in 3 columns) conveniently transformed in pial coordinates and ready-to-use.
4. The `classification_system` stating which units are neurons and which ones are not. 
5. The `cell_type` from the AIBS fine classification and including non-neuronal objects. 
6. The estimated `brain_area` for this object. 
7. The proofreading status for axon and dendrite.
8. The estimated `layer` in which the neuron is in, inferred from the AIBS classification.
9. If set up to work with functional data, the data includes whether if a neuron is functionally coregistrated or not. If not, all rows are set to `not_matched`.

The `process_nucleus_data` function returns the table of units and the start and end of each layer. Please notice that no information about synapses has been yet processed. **The table does filter for all multisoma objects, which are not included. Thus, all `nucleus_id` and `pt_root_id` are unique.** The only exception to this rule is when working with functional data, because the same neuron can appear in several scans. Please refer to the [section on functional data](#working-with-functional-data) below. 

For the moment we will focus on the connectomics. Let's download the nucleus information and build the unit table. 

In [32]:
#Download all data related to nucleus: reference table, various classifications, functional matchs
cleaner.download_nucleus_data()

#Process the data and obtain units and segment tables
units, segments = cleaner.process_nucleus_data()

Transform positions: 100%|██████████| 94014/94014 [00:00<00:00, 124155.53it/s]


In [6]:
units

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type
0,373879,864691136090135607,828.189723,638.553801,783.72,excitatory_neuron,6P-CT,V1,none,none,L6,not_matched
1,408486,864691135194387242,891.157407,662.693656,1002.96,nonneuron,oligo,V1,none,none,L6,not_matched
4,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
6,200523,864691135777840480,513.635721,509.463386,652.56,excitatory_neuron,6P-CT,V1,none,none,L6,not_matched
7,372649,864691136273924109,829.039163,557.615841,1039.92,excitatory_neuron,6P-IT,V1,none,none,L6,not_matched
...,...,...,...,...,...,...,...,...,...,...,...,...
94009,167679,864691135783968051,427.922269,496.375161,1005.96,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
94010,303490,864691135940617126,668.626983,461.348673,1003.64,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
94011,233088,864691135430122800,533.829149,454.887682,1007.28,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
94012,232979,864691135884866160,519.002564,451.791678,986.64,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched


In [7]:
segments

,layer,region_index,y_start,y_end,height
0,L1,0,1.224539,70.981459,69.756920
1,L2/3,0,70.981459,250.356396,179.374937
2,L4,0,250.356396,359.974413,109.618017
3,L5,0,359.974413,509.453527,149.479114
4,L6,0,509.453527,828.342305,318.888777


The next step is to download the synapses. Again, this operation is automated by the library. Downloading the whole dataset of synapses is very long. Thus, one can download synapses from a subset of pre- and post- neurons. 

As an example here, we will sample 120 random units and ask to download all potential presynaptic connections to those

In [8]:
postids = units['pt_root_id'].sample(n=120)
preids  = units['pt_root_id']

Now the download happens. Queries **to the API can be slow and sometimes need to retry.** To minimize problems, the downloader will try to download small chunks of `neur_per_steps=500` neurons to facilitate the task. If the downloader finds a problem and stops, it will automatically retry after `delay=5` seconds for `max_retries=10` times. The progress will be indicated and the progress bar will show the amount of succesfully downloaded chunks. In our example, we have just 120 neurons, meaning a single chunk (since `neur_per_steps=500`). 

You can select also if you want all synapses between neurons or just an effective synaptic volume between a pair of neurons. In the first case (`drop_synapses_duplicates=False`), you will download all individual synapses between the two neurons, their positions in space and synaptic volume; in the second one, (`drop_synapses_duplicates=True`, the default) you will get the sum of all synaptic volumes accross all synapses between those neurons. 

In [12]:
cleaner.download_synapse_data(preids, postids)

/home/victor/Fisica/Research/Milan/MICrONS-datacleaner/tutorial/../data/1300/raw/synapses


  0%|          | 0/120 [00:00<?, ?it/s]

500it [02:38,  3.15it/s]               


If the process fails for any reason (download trials exceeded, internet connection lost...) one can check how many chunks were downloaded inside the folder `data/raw/synapses/`. Look at the last `connections_table_X`. Then, you can call `cleaner.download_synapse_data(preids, postids, start_index=X-1)` to start the downloaded right where you left it.

Once the function above is successful, all the downloaded chunks can be merged into a single synapse table with

In [13]:
cleaner.merge_synapses(syn_table_name="example_merged_synapses")

In [14]:
synapses = pd.read_csv("../data/1300/raw/example_merged_synapses.csv")
synapses

,pre_pt_root_id,post_pt_root_id,size
0,864691134884743674,864691135989282304,6172
1,864691134884751866,864691135467530380,7072
2,864691134884791290,864691135582430317,2380
3,864691134884807418,864691135564651095,396
4,864691134885023994,864691135447593556,3304
...,...,...,...
13346,864691137198725953,864691136032443579,3468
13347,864691137198754369,864691136144334388,1868
13348,864691137198939713,864691136134404491,2792
13349,864691137198988097,864691135156573028,10040


Now you are ready to start analysing your data!

## Using filters 

One of the first we can do is to filter the data. In fact, it is advisable to carefully filter the units to study before downloading the synapses between them. The package includes some functions with pre-defined queries to filter the data with ease. We will start by importing the filters subpackage

In [15]:
import microns_datacleaner.filters as fl

With the filter functions, we can, obtain all neurons that are in V1 **and**  L5:

In [16]:
units_filter = fl.filter_neurons(units, layer='L5', brain_area='V1')
units_filter.head()

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type
4,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
28,161434,864691135570002310,448.030669,392.837733,703.84,excitatory_neuron,5P-IT,V1,none,none,L5,not_matched
29,196209,864691136084039276,458.937273,417.241168,684.40,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
30,165545,864691135538697586,451.910507,416.048206,1030.32,inhibitory_neuron,MC,V1,none,none,L5,not_matched
38,227994,864691136309636954,569.805827,360.897595,800.04,excitatory_neuron,4P,V1,none,none,L5,not_matched


Or all the neurons at layers 2/3, 4 or 5

In [17]:
units_filter = fl.filter_neurons(units, layer=['L2/3', 'L4', 'L5'])
units_filter.head()

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type
4,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,not_matched
19,152813,864691136239314236,468.347072,147.402469,633.60,nonneuron,astrocyte,V1,none,none,L2/3,not_matched
20,190151,864691135275621605,524.797840,223.267165,719.96,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_extended,L2/3,not_matched
22,189115,864691135738006641,501.233729,104.152149,960.08,excitatory_neuron,23P,V1,none,none,L2/3,not_matched
23,156155,864691135213116544,464.644371,213.957017,1093.04,inhibitory_neuron,MC,V1,none,none,L2/3,not_matched


Finally, let's get all neurons in layers 2/3 or 4, that are in V1 or RL and have some kind of proofreading in the axons:

In [18]:
units_filter = fl.filter_neurons(units, layer=['L2/3', 'L4'], proofread='ax_clean', brain_area=['V1', 'RL'])
units_filter.head()

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type
20,190151,864691135275621605,524.797840,223.267165,719.96,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_extended,L2/3,not_matched
86,223016,864691135763941174,613.394783,104.264055,927.88,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_extended,L2/3,not_matched
87,222257,864691136488446226,547.738750,131.413065,711.88,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_extended,L2/3,not_matched
196,258319,864691136021936376,660.944993,236.334898,849.52,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_clean,L2/3,not_matched
274,260552,864691135784109363,651.860363,294.645008,840.08,excitatory_neuron,4P,V1,axon_partially_extended,dendrite_clean,L4,not_matched


This allows one to filter complex combinations with a single line. However, it is not the end of the story. It is possible to do the same for synapses, by indicating the filters separately for the pre and post- synaptic neurons, respectively. For example, let's find all the connections that originate from V1 from the ones we downloaded. Observe the use of `None` to indicate that there should be no filter whatsoever in the postsynaptic neurons' brain area: 

In [19]:
syn_filter = fl.filter_connections(units, synapses,  brain_area=['V1', None]) 
syn_filter.head()

,pre_pt_root_id,post_pt_root_id,size
0,864691134884743674,864691135989282304,6172
1,864691134884751866,864691135467530380,7072
2,864691134884791290,864691135582430317,2380
4,864691134885023994,864691135447593556,3304
5,864691134885140218,864691136521856401,9960


We can check which ones of those do not end in V1

In [20]:
syn_filter = fl.filter_connections(units, synapses,  brain_area=['V1', ['RL', 'AL', 'LM']]) 
syn_filter.head()

,pre_pt_root_id,post_pt_root_id,size
39,864691134886347002,864691134917958922,344
40,864691134886347002,864691135942090612,15488
42,864691134886347002,864691136907704942,3976
120,864691134940425315,864691135847940446,4776
132,864691134941088867,864691135847940446,1780


But all those connections are probably problematic. We should stick, as a rule of thomb, to those that have proofread axons:

In [21]:
syn_filter = fl.filter_connections(units, synapses,  brain_area=['V1', ['RL', 'AL', 'LM']], proofread=['ax_clean', None]) 
syn_filter.head()

,pre_pt_root_id,post_pt_root_id,size
190,864691134948973308,864691135397121057,12164
297,864691134989909114,864691135807342621,4924
508,864691135066892868,864691135508857609,2520
622,864691135101431072,864691136032443579,3080
1095,864691135209432185,864691136023793721,41348


There are ways to be more flexible. For example, it is possible to inspect all the connections that have certain `pt_root_id` as pre or post synaptic neurons. For example, let's inspect the first two neurons in the table above:

In [22]:
syn_filter = fl.synapses_by_id(synapses, pre_ids=[864691134949221372, 864691134990291066]) 
syn_filter

,pre_pt_root_id,post_pt_root_id,size
194,864691134949221372,864691135719157041,16280
195,864691134949221372,864691136925340490,16908


In [23]:
syn_filter = fl.synapses_by_id(synapses, post_ids=[864691135082715127, 864691135510437328]) 
syn_filter

,pre_pt_root_id,post_pt_root_id,size


It is also possible to ask for all connections between the given pre and post ids. For example, in our subset of units do we have more connections from V1 to AL or from AL to V1? 

In [24]:
v1_neurons = fl.filter_neurons(units, brain_area='V1')
al_neurons = fl.filter_neurons(units, brain_area='AL')

v1toal = len(fl.synapses_by_id(synapses, pre_ids=v1_neurons['pt_root_id'], post_ids=al_neurons['pt_root_id'])) 
altov1 = len(fl.synapses_by_id(synapses, pre_ids=al_neurons['pt_root_id'], post_ids=v1_neurons['pt_root_id'])) 

f"V1 to AL: {v1toal};   AL to V1: {altov1}"

'V1 to AL: 19;   AL to V1: 10'

Also, how many connections do we have that include at least a presynaptic neuron in V1 **or** a postsynaptic neuron in AL? Observe that the following code only cares about fulfilling the condition for pre and post separately. So if a presynaptic neuron is in V1, and the postsynaptic is in RL, is also valid. This is done with the `both` keyword

In [25]:
len(fl.synapses_by_id(synapses, pre_ids=v1_neurons['pt_root_id'], post_ids=al_neurons['pt_root_id'], both=False)) 

10871

There are also convenience function to obtain all the inputs to / outputs from a particular neuron

## Working with functional data

The functional part of the data is in a different dataset, and hence the responses cannot be downloaded using the Microns DataCleaner interface. However, we provide average firing rates (with their standard errors) for each one of the presented directions of the Monet2 stimuli, as well as code to fit the tuning curves for orientation and direction. 

Additionally, the MICrONS consortium includes a table with funcitonal properties for neurons obtained using a Digital Twin trained on the whole response data. 

If none of this is of interest to you, and you want to work with the raw functional database, the DataCleaner package can still fill the session, scan ID and unit ID of the coregistration, in order to match your analysis with the unit table generated by the package. Let's go one by one.

### Coregistration in the unit table

Imagine that we just want to know which neurons were coregistrated, as well as their identifiers to match them with the functional data. So far, when we called `process_nucleus_data`, we did not ask for any extra information on functional data. But we can load the unit table indicating that we want the functionally matched neurons

In [26]:
units, segments = cleaner.process_nucleus_data(functional_data='match')

Transform positions: 100%|██████████| 94014/94014 [00:00<00:00, 123717.26it/s]


In [27]:
fl.filter_neurons(units, tuning='matched')

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,session,scan_idx,functional_unit_id,tuning_type
15,190151,864691135275621605,524.797840,223.267165,719.96,excitatory_neuron,23P,V1,axon_partially_extended,dendrite_extended,L2/3,4.0,7.0,3229.0,matched
16,189115,864691135738006641,501.233729,104.152149,960.08,excitatory_neuron,23P,V1,none,none,L2/3,6.0,2.0,2184.0,matched
20,153275,864691135463256477,476.232006,135.114928,804.24,excitatory_neuron,23P,V1,none,none,L2/3,6.0,2.0,1890.0,matched
21,153085,864691135748985641,475.434732,106.777609,743.60,excitatory_neuron,23P,V1,none,none,L2/3,6.0,7.0,1525.0,matched
31,227619,864691135387453569,568.397693,297.686330,748.12,excitatory_neuron,4P,V1,none,none,L4,9.0,3.0,2116.0,matched
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94019,402179,864691135082235383,899.999179,481.591117,1037.12,excitatory_neuron,5P-ET,V1,none,none,L5,7.0,4.0,12181.0,matched
94023,560779,864691135477491368,1216.787835,468.382806,847.20,excitatory_neuron,5P-ET,RL,none,none,L5,7.0,4.0,12106.0,matched
94025,402104,864691135927368462,897.947642,466.121447,1004.00,excitatory_neuron,5P-ET,V1,none,none,L5,5.0,3.0,7357.0,matched
94026,402104,864691135927368462,897.947642,466.121447,1004.00,excitatory_neuron,5P-ET,V1,none,none,L5,7.0,4.0,12434.0,matched


We see that the `tuning_type` of these neurons is `matched`. They have their `session`, `scan_idx` and `functional_unit_id`, which allows to search them in the functional data. Observe that `functional_unit_id` is what the functional data labels just as `unit_id`. Here we are more specific to avoid confusion with other identifiers used for the connectomics. Non-matched neurons do have NaN values for those columns.

Observe that some neurons are scanned several times, and hence leads to repeated rows, including repeated `nucleus_id`:


In [28]:
units['nucleus_id'].value_counts()

nucleus_id
582425    5
553492    4
422300    4
582010    4
487505    4
         ..
550916    1
201628    1
154696    1
669727    1
267033    1
Name: count, Length: 90303, dtype: int64

In [29]:
#Show all sessions and scans where this neuron was found
units[units['nucleus_id'] == 582425]

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,session,scan_idx,functional_unit_id,tuning_type
18571,582425,864691135492201823,1284.155437,202.843824,865.88,excitatory_neuron,23P,RL,none,none,L2/3,4.0,7.0,2650.0,matched
18572,582425,864691135492201823,1284.155437,202.843824,865.88,excitatory_neuron,23P,RL,none,none,L2/3,7.0,4.0,6039.0,matched
18573,582425,864691135492201823,1284.155437,202.843824,865.88,excitatory_neuron,23P,RL,none,none,L2/3,7.0,5.0,4046.0,matched
18574,582425,864691135492201823,1284.155437,202.843824,865.88,excitatory_neuron,23P,RL,none,none,L2/3,8.0,5.0,1553.0,matched
18575,582425,864691135492201823,1284.155437,202.843824,865.88,excitatory_neuron,23P,RL,none,none,L2/3,8.0,5.0,4463.0,matched


The data coming from the Digital Twin also features multiscan properties for some neurons. While it is possible to work with the full multiscan information, 

In [30]:
units, segments = cleaner.process_nucleus_data(functional_data='all')
fl.filter_neurons(units, tuning='matched')

Transform positions: 100%|██████████| 94014/94014 [00:00<00:00, 119732.68it/s]


,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,session,scan_idx,functional_unit_id,pref_ori,pref_dir,gOSI,gDSI,cc_abs,tuning_type
16,189115,864691135738006641,501.233729,104.152149,960.08,excitatory_neuron,23P,V1,none,none,L2/3,6.0,2.0,2184.0,0.914503,0.914503,0.255712,0.031162,0.379552,matched
20,153275,864691135463256477,476.232006,135.114928,804.24,excitatory_neuron,23P,V1,none,none,L2/3,6.0,2.0,1890.0,2.010759,5.152352,0.084251,0.030395,0.526770,matched
21,153085,864691135748985641,475.434732,106.777609,743.60,excitatory_neuron,23P,V1,none,none,L2/3,6.0,7.0,1525.0,2.616992,2.616992,0.129408,0.056684,0.208270,matched
31,227619,864691135387453569,568.397693,297.686330,748.12,excitatory_neuron,4P,V1,none,none,L4,9.0,3.0,2116.0,2.374382,2.374382,0.282935,0.093646,0.254217,matched
34,363874,864691135699950114,851.072250,316.056947,942.16,excitatory_neuron,4P,V1,none,none,L4,7.0,3.0,4674.0,0.146806,0.146806,0.135074,0.106339,0.450477,matched
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91726,618622,864691136009507502,1340.481785,482.802315,1007.04,excitatory_neuron,5P-ET,RL,none,none,L5,4.0,7.0,8206.0,1.441359,4.582952,0.128425,0.015605,0.228502,matched
91739,303557,864691135213195392,659.127328,465.657119,1043.04,excitatory_neuron,5P-ET,V1,none,none,L5,7.0,3.0,9069.0,0.063529,3.205122,0.324385,0.057091,0.502292,matched
91746,369945,864691135375228745,798.635712,449.851932,853.00,excitatory_neuron,5P-ET,V1,none,none,L5,7.0,3.0,8708.0,0.021779,3.163372,0.219530,0.055386,0.416705,matched
91747,302754,864691135114540953,652.650672,483.143181,801.92,excitatory_neuron,5P-ET,V1,none,none,L5,4.0,7.0,8085.0,3.125382,3.125382,0.306179,0.045851,0.845627,matched


there is another possibility, which is to keep only the scan with the largest `cc_abs`, i.e., the scans that are better predicted by the Digital Twin

In [58]:
units, segments = cleaner.process_nucleus_data(functional_data='best_only')
fl.filter_neurons(units, tuning='matched')

Transform positions: 100%|██████████| 94014/94014 [00:00<00:00, 143580.34it/s]


,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,pref_ori,pref_dir,gOSI,gDSI,cc_abs,tuning_type
16,189115,864691135738006641,501.233729,104.152149,960.08,excitatory_neuron,23P,V1,none,none,L2/3,0.914503,0.914503,0.255712,0.031162,0.379552,matched
20,153275,864691135463256477,476.232006,135.114928,804.24,excitatory_neuron,23P,V1,none,none,L2/3,2.010759,5.152352,0.084251,0.030395,0.526770,matched
21,153085,864691135748985641,475.434732,106.777609,743.60,excitatory_neuron,23P,V1,none,none,L2/3,2.616992,2.616992,0.129408,0.056684,0.208270,matched
31,227619,864691135387453569,568.397693,297.686330,748.12,excitatory_neuron,4P,V1,none,none,L4,2.374382,2.374382,0.282935,0.093646,0.254217,matched
34,363874,864691135699950114,851.072250,316.056947,942.16,excitatory_neuron,4P,V1,none,none,L4,0.146806,0.146806,0.135074,0.106339,0.450477,matched
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90266,618622,864691136009507502,1340.481785,482.802315,1007.04,excitatory_neuron,5P-ET,RL,none,none,L5,1.441359,4.582952,0.128425,0.015605,0.228502,matched
90279,303557,864691135213195392,659.127328,465.657119,1043.04,excitatory_neuron,5P-ET,V1,none,none,L5,0.063529,3.205122,0.324385,0.057091,0.502292,matched
90286,369945,864691135375228745,798.635712,449.851932,853.00,excitatory_neuron,5P-ET,V1,none,none,L5,0.021779,3.163372,0.219530,0.055386,0.416705,matched
90287,302754,864691135114540953,652.650672,483.143181,801.92,excitatory_neuron,5P-ET,V1,none,none,L5,3.125382,3.125382,0.306179,0.045851,0.845627,matched


So approximately 2000 entries of the previous table where related to multiscans. This new table has unique `nucleus_id`, as all multiscans were taken out. Observe that this table does not give information about session, scan or unit id. 

Finally, observe that the Digital Twin displays less neurons that the `matched` mode, even when all multiscan are used. This is because the Digital Twin is trained on the V3 of the manually coregistrated neurons, while the `matched` one uses the V4. In fact, if we want even more coregistrated neurons, it is possible to use the automatic coregistration procedure, but it is not manually checked. 

## Merging other tables

Sometimes, we would like to have more information on our unit table. Maybe a different classification scheme or custom analysis from the funcitonal data. The DataCleaner has a helper function to merge the tables with ease on the correct IDs. 

We will show two examples here. First, we merge add the automatic coregistration table that we downloaded before to our unit table. Although we can use `pd.read_csv`, it is simpler to use our convenience function `read_table`, which also reads the table from the current used version.

In [6]:
auto_coreg = cleaner.read_table("coregistration_auto_phase3_fwd_apl_vess_combined_v2")
auto_coreg.head()

,id_ref,created_ref,valid_ref,volume,pt_position_x,pt_position_y,pt_position_z,bb_start_position_x,bb_start_position_y,bb_start_position_z,...,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score
0,388868,2020-09-28 22:44:53.222573+00:00,t,292.632003,238400,102576,15126,NaN,NaN,NaN,...,1,2024-12-18 19:00:18.303101+00:00,t,388868,4,7,107,1,20.073000,3.133400
1,357165,2020-09-28 22:44:28.479993+00:00,t,254.621450,218304,103136,15112,NaN,NaN,NaN,...,2,2024-12-18 19:00:18.303788+00:00,t,357165,4,7,152,1,15.722900,14.285700
2,323782,2020-09-28 22:45:16.004514+00:00,t,354.078392,209152,102896,15206,NaN,NaN,NaN,...,3,2024-12-18 19:00:18.304440+00:00,t,323782,4,7,487,1,24.411051,7.901162
3,452040,2020-09-28 22:45:19.539072+00:00,t,374.225306,264992,99632,15169,NaN,NaN,NaN,...,4,2024-12-18 19:00:18.305077+00:00,t,452040,4,7,600,1,21.356300,0.270600
4,485026,2020-09-28 22:44:53.278972+00:00,t,292.830085,278880,100192,15127,NaN,NaN,NaN,...,5,2024-12-18 19:00:18.305662+00:00,t,485026,4,7,636,1,18.895700,2.232600


In [18]:
#Without functional info
units, segments = cleaner.process_nucleus_data()

Transform positions: 100%|██████████| 94014/94014 [00:00<00:00, 131039.46it/s]


Then, we can use `merge_table` in order to add the new information. Although it is possible to do the process manually with the Pandas function `units.merge()`, `merge_table` stremalines the process to avoid some common pitfalls. In Microns, there are several IDs, which sometimes can lead to errors.  

The method `merge_table` leverages the fact that our unit table works with the `nucleus_id`, which comes from the nucleus reference and it is independent different versions of the dataset -which is not guaranteed for `pt_root_id`. By default, `method=nucleus_id`. In this mode, it uses the `target_id` present in many tables to add the new columns to the table. One can also select which columns are added to the table.

In [19]:
#Add the coordinates, as well as the separation score to measure how 'trustable' the unit is
units = cleaner.merge_table(units, auto_coreg, columns=['session', 'scan_idx', 'unit_id', 'score'])

#Respect our convention that unit_id is functional_unit_id, which can be useful for future merges
units = units.rename(columns={'unit_id' : 'functional_unit_id'})

#Flag the matched neurons accordingly: if session is 'not nan', then they are matched 
units.loc[units['session'].notna(), 'tuning_type'] = 'matched' 
units

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type,session,scan_idx,functional_unit_id,score
0,373879,864691136090135607,828.189723,638.553801,783.72,excitatory_neuron,6P-CT,V1,none,none,L6,not_matched,NaN,NaN,NaN,NaN
1,408486,864691135194387242,891.157407,662.693656,1002.96,nonneuron,oligo,V1,none,none,L6,not_matched,NaN,NaN,NaN,NaN
2,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,matched,8.0,4.0,9746.0,4.537519
3,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,matched,8.0,5.0,10096.0,2.701442
4,200523,864691135777840480,513.635721,509.463386,652.56,excitatory_neuron,6P-CT,V1,none,none,L6,matched,6.0,2.0,8491.0,5.347090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136567,232979,864691135884866160,519.002564,451.791678,986.64,excitatory_neuron,5P-ET,V1,none,none,L5,matched,5.0,3.0,7131.0,2.356100
136568,232979,864691135884866160,519.002564,451.791678,986.64,excitatory_neuron,5P-ET,V1,none,none,L5,matched,7.0,3.0,9143.0,5.833800
136569,232979,864691135884866160,519.002564,451.791678,986.64,excitatory_neuron,5P-ET,V1,none,none,L5,matched,7.0,4.0,12067.0,-1.607884
136570,232979,864691135884866160,519.002564,451.791678,986.64,excitatory_neuron,5P-ET,V1,none,none,L5,matched,8.0,7.0,6919.0,7.088232


Observe that `merge_table` also takes care of eliminating duplicated columns that may be present in both merging tables. Pandas' `merge` generally creates two columns with suffixes `_x` and `_y` to disambiguate. In our case, those columns would be equal, as they refer to the same unit. Thus, one of them can be dropped safely. Additionally, `merge_table` has a `how` keyword, that acts the same way as Pandas'. By default, `how=left`, since the main use of the method is to add new columns to our unit table.  

Another way of merging information is using `method='functional'`. This will match by `session`, `scan_idx`, and functional `unit_id`, adding the functional properties of the data to our unit table.  **For this method to work, it is important that the functional `unit_id` column in the unit table is called `functional_unit_id`.** This will be the case when using the functional coregistrations generated by the package, but if coregistration tables are merged by hand, as in this case, they have to be renamed. There is no need to touch the incoming table.

We will now add the measured preferred orientation for these neurons, by using the `functional_properties` table provided by us. We will include the rates for orientation as columns in the table: 

In [21]:
func_props = pd.read_csv("../data/processed/functional/functional_fits.csv") 

units = cleaner.merge_table(units, func_props, columns=['rate_ori'], method='functional')
units.head()

,nucleus_id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,brain_area,strategy_axon,strategy_dendrite,layer,tuning_type,session,scan_idx,functional_unit_id,score,rate_ori
0,373879,864691136090135607,828.189723,638.553801,783.72,excitatory_neuron,6P-CT,V1,none,none,L6,not_matched,NaN,NaN,NaN,NaN,NaN
1,408486,864691135194387242,891.157407,662.693656,1002.96,nonneuron,oligo,V1,none,none,L6,not_matched,NaN,NaN,NaN,NaN,NaN
2,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,matched,8.0,4.0,9746.0,4.537519,NaN
3,199883,864691136041571414,514.440335,481.908658,1044.32,excitatory_neuron,5P-ET,V1,none,none,L5,matched,8.0,5.0,10096.0,2.701442,[4.39174989 6.05327217 4.90589327 6.1549597 6...
4,200523,864691135777840480,513.635721,509.463386,652.56,excitatory_neuron,6P-CT,V1,none,none,L6,matched,6.0,2.0,8491.0,5.347090,[1.04788442 0.51756797 0.29822768 0.39735019 0...


## Working with rates and reindex

...to do